### Import Dependencies

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
from sklearn.metrics import confusion_matrix, classification_report

### Prep Data

In [2]:
# Read in data
df = pd.read_csv("resources/all_data.csv")

# Set index to quarter
df = df.set_index('quarter')

In [3]:
# Save 2019 Q1 & Q2
df_2019 = df.iloc[[-4,-3],:]
df_2019 = df_2019.drop(columns=['recession_actual'])
df_2019

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,
2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,4.133333,2.401311,0.083088,0.004646,0.170000,-0.271429,4.5,1.6,104.40334,3783.364,1.458,313000.0,14266.250
2019Q2,255.139333,21340.267,4.7,1255.666667,0.828815,3.500000,2.397813,-0.001457,0.024002,0.213333,0.254902,2.4,1.6,103.20060,3749.471,1.457,322500.0,14511.176


In [4]:
# Drop rows with missing values
df = df.dropna()
df.tail()

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,recession_actual,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,,
2018Q1,249.250333,20163.159,5.0,1320.666667,0.202456,0.0,4.333333,1.448966,0.204683,0.083902,0.596667,-0.113861,6.9,1.8,104.59493,3542.412,1.451,331800.0,13728.357
2018Q2,250.578667,20510.177,7.1,1259.666667,0.589182,0.0,3.833333,1.727176,0.192007,0.075492,0.446667,-0.251397,2.7,2.0,103.33928,3561.592,1.461,315600.0,13939.828
2018Q3,251.828667,20749.752,4.8,1233.000000,0.821959,0.0,3.866667,1.923492,0.113663,0.047184,0.253333,-0.432836,3.3,2.0,103.69309,3683.981,1.462,330900.0,14114.559
2018Q4,252.759000,20897.804,2.9,1185.000000,0.592021,0.0,3.566667,2.217097,0.152641,0.066218,0.233333,-0.078947,2.8,1.9,105.15026,3725.234,1.462,322800.0,14211.920
2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,0.0,4.133333,2.401311,0.083088,0.004646,0.170000,-0.271429,4.5,1.6,104.40334,3783.364,1.458,313000.0,14266.250


### Shift Data

In [5]:
df['recession_1q_out'] = df['recession_actual'].shift(-1)
df['recession_2q_out'] = df['recession_actual'].shift(-2)
df['recession_4q_out'] = df['recession_actual'].shift(-4)

In [6]:
# Create three datasets -- 1 for each model (recession 1Qtr out, 2Qtrs out, 4Qtrs out)
df_q1 = df.drop(columns=['recession_2q_out','recession_4q_out','recession_actual'])
df_q2 = df.drop(columns=['recession_4q_out','recession_1q_out','recession_actual'])
df_q4 = df.drop(columns=['recession_1q_out','recession_2q_out','recession_actual'])

In [7]:
# Delete missing values
df_q1 = df_q1.dropna()
df_q2 = df_q2.dropna()
df_q4 = df_q4.dropna()

In [8]:
# Define y variables
y1 = df_q1['recession_1q_out']
y2 = df_q2['recession_2q_out']
y3 = df_q4['recession_4q_out']

In [9]:
# Define X
X1 = df_q1.drop(columns=['recession_1q_out'])
X2 = df_q2.drop(columns=['recession_2q_out'])
X3 = df_q4.drop(columns=['recession_4q_out'])

### Split and scale data

In [10]:
# Split data into training and testing
X1_train, X1_test, y1_train, y1_test=train_test_split(X1, y1, train_size=0.8, stratify=y1, shuffle=False)
X2_train, X2_test, y2_train, y2_test=train_test_split(X2, y2, train_size=0.8, stratify=y2, shuffle=False)
X3_train, X3_test, y3_train, y3_test=train_test_split(X3, y3, train_size=0.8, stratify=y3, shuffle=False)

In [11]:
# Create scaler object
X1_scaler = StandardScaler().fit(X1_train)
X2_scaler = StandardScaler().fit(X2_train)
X3_scaler = StandardScaler().fit(X3_train)

In [12]:
# Scale training data
X1_train_scaled = X1_scaler.transform(X1_train)
X2_train_scaled = X2_scaler.transform(X2_train)
X3_train_scaled = X3_scaler.transform(X3_train)

# Scale testing data
X1_test_scaled = X1_scaler.transform(X1_test)
X2_test_scaled = X2_scaler.transform(X2_test)
X3_test_scaled = X3_scaler.transform(X3_test)

# Scale full X data (no splits)
X1_full_scaled = X1_scaler.transform(X1)
X2_full_scaled = X2_scaler.transform(X2)
X3_full_scaled = X3_scaler.transform(X3)

### Reshape data to fit LSTM format

In [13]:
# Method to reshape data
def reshape_data(obj):
    reshaped_obj = np.reshape(obj, (obj.shape[0], obj.shape[1], 1))
    return reshaped_obj

In [14]:
# Reshape training data
reshaped_X1_train_scaled = reshape_data(X1_train_scaled)
reshaped_X2_train_scaled = reshape_data(X2_train_scaled)
reshaped_X3_train_scaled = reshape_data(X3_train_scaled)

In [15]:
# Reshape testing data
reshaped_X1_test_scaled = reshape_data(X1_test_scaled)
reshaped_X2_test_scaled = reshape_data(X2_test_scaled)
reshaped_X3_test_scaled = reshape_data(X3_test_scaled)

In [16]:
# Reshape X_full
reshaped_X1_full = reshape_data(X1_full_scaled)
reshaped_X2_full = reshape_data(X2_full_scaled)
reshaped_X3_full = reshape_data(X3_full_scaled)

# Build Model

In [17]:
# Initialize model
model = Sequential()

In [18]:
# Add layers
model.add(LSTM(128, input_shape=(reshaped_X1_train_scaled.shape[1],1), return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())  # Normalize activation outputs

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [19]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

## Train and predict on X1-Y1 data (recession 1 quarter out)

In [20]:
# Fit the model to the training data
model.fit(reshaped_X1_train_scaled, y1_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 108 samples, validate on 28 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
108/108 - 15s - loss: 0.9774 - acc: 0.5093 - val_loss: 0.6744 - val_acc: 0.8929
Epoch 2/100
108/108 - 1s - loss: 0.8246 - acc: 0.5556 - val_loss: 0.6569 - val_acc: 0.8929
Epoch 3/100
108/108 - 1s - loss: 0.6435 - acc: 0.6481 - val_loss: 0.6391 - val_acc: 0.8929
Epoch 4/100
108/108 - 1s - loss: 0.6706 - acc: 0.6574 - val_loss: 0.6238 - val_acc: 0.8929
Epoch 5/100
108/108 - 1s - loss: 0.6344 - acc: 0.6944 - val_loss: 0.6030 - val_acc: 0.8929
Epoch 6/100
108/108 - 1s - loss: 0.6754 - acc: 0.7037 - val_loss: 0.5841 - val_acc: 0.8929
Epoch 7/100
108/108 - 1s - loss: 0.6223 - acc: 0.7130 - val_loss: 0.5673 - val_acc: 0.8929
Epoch 8/100
108/108 - 2s - loss: 0.5199 - acc: 0.7593 - val_loss: 0.5521 - val_acc: 0.8929
Epoch 9/100
108/108 - 1s - loss: 0.4724 - acc: 0.7778 - val_loss: 0.5356 - val_acc: 0.8929
Epoch 10/100
108/108 - 1s - loss: 0.4157 

108/108 - 1s - loss: 0.2473 - acc: 0.9259 - val_loss: 0.5410 - val_acc: 0.8571
Epoch 87/100
108/108 - 1s - loss: 0.1957 - acc: 0.9074 - val_loss: 0.5234 - val_acc: 0.8571
Epoch 88/100
108/108 - 1s - loss: 0.1752 - acc: 0.9352 - val_loss: 0.5285 - val_acc: 0.8571
Epoch 89/100
108/108 - 1s - loss: 0.1691 - acc: 0.9352 - val_loss: 0.5453 - val_acc: 0.8571
Epoch 90/100
108/108 - 1s - loss: 0.1827 - acc: 0.9352 - val_loss: 0.5531 - val_acc: 0.8929
Epoch 91/100
108/108 - 2s - loss: 0.1409 - acc: 0.9259 - val_loss: 0.5634 - val_acc: 0.8929
Epoch 92/100
108/108 - 1s - loss: 0.1561 - acc: 0.9444 - val_loss: 0.5825 - val_acc: 0.8929
Epoch 93/100
108/108 - 1s - loss: 0.1901 - acc: 0.9074 - val_loss: 0.5821 - val_acc: 0.8929
Epoch 94/100
108/108 - 1s - loss: 0.1899 - acc: 0.9167 - val_loss: 0.5782 - val_acc: 0.8929
Epoch 95/100
108/108 - 1s - loss: 0.1380 - acc: 0.9444 - val_loss: 0.5775 - val_acc: 0.8929
Epoch 96/100
108/108 - 2s - loss: 0.2028 - acc: 0.9259 - val_loss: 0.5812 - val_acc: 0.8571
E

In [21]:
# Evaluate model using test data
model_loss1, model_accuracy1 = model.evaluate(reshaped_X1_test_scaled, y1_test, verbose=2)

34/34 - 0s - loss: 0.6491 - acc: 0.8529


In [22]:
# Make predictions using test data
predictions1_class = model.predict_classes(reshaped_X1_test_scaled) # Predicted class

In [23]:
# Compare results
one_qtr_out = pd.DataFrame({"Predicted":predictions1_class, "Actual":y1_test})
one_qtr_out.loc[one_qtr_out["Actual"]==1]

,Predicted,Actual
quarter,,
1982Q1,0,1.0
1979Q2,0,1.0
1990Q1,0,1.0
1981Q1,0,1.0
2008Q2,0,1.0


#### Confusion Matrix on X1-Y1 data (recession 1 quarter out)

In [24]:
# Create confusion matrix on X1 model
con_mat = confusion_matrix(y1_test, predictions1_class)
print(con_mat)

[[29  0]
 [ 5  0]]


In [25]:
# Score model
print(classification_report(y1_test, predictions1_class))

              precision    recall  f1-score   support

         0.0       0.85      1.00      0.92        29
         1.0       0.00      0.00      0.00         5

    accuracy                           0.85        34
   macro avg       0.43      0.50      0.46        34
weighted avg       0.73      0.85      0.79        34



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
# Save model
# name1 = f"unshuffled-1q-out-{int(time.time())}"
# model.save(f"models/{name1}.h5")

### Predict on 2019

In [27]:
# Scale 2019 data 
scaled_X1_2019 = X1_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X1_2019 = reshape_data(scaled_X1_2019)

# Predict on 2019
pred_X1_2019 = model.predict_proba(reshaped_X1_2019)
print(f"2019Q1 No Recession Probability: {pred_X1_2019[0][0]}")
print(f"2019Q1 Recession Probability: {pred_X1_2019[0][1]}")
print(f"2019Q2 No Recession Probability: {pred_X1_2019[1][0]}")
print(f"2019Q2 Recession Probability: {pred_X1_2019[1][1]}")

2019Q1 No Recession Probability: 0.9999264478683472
2019Q1 Recession Probability: 7.349427323788404e-05
2019Q2 No Recession Probability: 0.9990673661231995
2019Q2 Recession Probability: 0.0009326547151431441


#### Predict on full X1

In [28]:
pred_X1_full = model.predict_classes(reshaped_X1_full)

# Preview results
X1_full_results = pd.DataFrame({"Predicted":pred_X1_full, "Actual":y1})
X1_full_results.loc[X1_full_results["Actual"]==1]

# Export results for graphing
# X1_full_results.to_csv(f"resources/predictions/X1_NS_{int(time.time())}.csv")

,Predicted,Actual
quarter,,
1979Q1,0,1.0
1979Q2,0,1.0
1979Q3,0,1.0
1979Q4,0,1.0
1980Q1,0,1.0
1981Q1,0,1.0
1981Q2,0,1.0
1981Q3,0,1.0
1981Q4,0,1.0


### Train and predict on X2-Y2 data (recession 2 quarters out)

In [29]:
# Fit the model to the training data
model.fit(reshaped_X2_train_scaled, y2_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 108 samples, validate on 27 samples
Epoch 1/100
108/108 - 2s - loss: 0.4527 - acc: 0.8611 - val_loss: 0.6642 - val_acc: 0.8148
Epoch 2/100
108/108 - 2s - loss: 0.4166 - acc: 0.8241 - val_loss: 0.7679 - val_acc: 0.8148
Epoch 3/100
108/108 - 2s - loss: 0.3452 - acc: 0.8611 - val_loss: 0.9215 - val_acc: 0.8148
Epoch 4/100
108/108 - 2s - loss: 0.2915 - acc: 0.9259 - val_loss: 1.0455 - val_acc: 0.8148
Epoch 5/100
108/108 - 2s - loss: 0.3056 - acc: 0.8796 - val_loss: 1.1205 - val_acc: 0.8148
Epoch 6/100
108/108 - 2s - loss: 0.4008 - acc: 0.8519 - val_loss: 1.0918 - val_acc: 0.8148
Epoch 7/100
108/108 - 3s - loss: 0.2502 - acc: 0.8704 - val_loss: 0.9583 - val_acc: 0.8148
Epoch 8/100
108/108 - 3s - loss: 0.3542 - acc: 0.8889 - val_loss: 0.8501 - val_acc: 0.8148
Epoch 9/100
108/108 - 3s - loss: 0.3900 - acc: 0.8889 - val_loss: 0.7917 - val_acc: 0.8148
Epoch 10/100
108/108 - 4s - loss: 0.2804 - acc: 0.8796 - val_loss: 0.7886 - val_acc: 0.8148
Epoch 11/100
108/108 - 3s - loss: 0.3071 - a

Epoch 90/100
108/108 - 2s - loss: 0.1377 - acc: 0.9537 - val_loss: 0.0235 - val_acc: 1.0000
Epoch 91/100
108/108 - 2s - loss: 0.0674 - acc: 0.9630 - val_loss: 0.1012 - val_acc: 0.9630
Epoch 92/100
108/108 - 1s - loss: 0.1509 - acc: 0.9537 - val_loss: 0.0614 - val_acc: 0.9630
Epoch 93/100
108/108 - 2s - loss: 0.1113 - acc: 0.9352 - val_loss: 0.0485 - val_acc: 1.0000
Epoch 94/100
108/108 - 2s - loss: 0.0725 - acc: 0.9722 - val_loss: 0.0554 - val_acc: 0.9630
Epoch 95/100
108/108 - 2s - loss: 0.0982 - acc: 0.9722 - val_loss: 0.0528 - val_acc: 1.0000
Epoch 96/100
108/108 - 2s - loss: 0.0587 - acc: 0.9907 - val_loss: 0.0371 - val_acc: 1.0000
Epoch 97/100
108/108 - 2s - loss: 0.0898 - acc: 0.9815 - val_loss: 0.0238 - val_acc: 1.0000
Epoch 98/100
108/108 - 2s - loss: 0.0785 - acc: 0.9630 - val_loss: 0.0476 - val_acc: 1.0000
Epoch 99/100
108/108 - 2s - loss: 0.0596 - acc: 0.9722 - val_loss: 0.0456 - val_acc: 1.0000
Epoch 100/100
108/108 - 2s - loss: 0.0806 - acc: 0.9722 - val_loss: 0.0302 - val

In [30]:
# Evaluate model using test data
model_loss2, model_accuracy2 = model.evaluate(reshaped_X2_test_scaled, y2_test, verbose=2)

34/34 - 0s - loss: 0.7560 - acc: 0.8235


In [31]:
# Make predictions using test data
predictions2_class = model.predict_classes(reshaped_X2_test_scaled)

In [32]:
# Compare results
two_qtrs_out = pd.DataFrame({"Predicted":predictions2_class, "Actual":y2_test})
two_qtrs_out.loc[two_qtrs_out["Actual"]==1]

,Predicted,Actual
quarter,,
1981Q4,0,1.0
1979Q1,1,1.0
1979Q4,0,1.0
1990Q3,0,1.0
1980Q4,1,1.0


#### Confusion Matrix on X2-Y2 data (recession 2 quarters out)

In [33]:
# Create confusion matrix on X2 model
con_mat = confusion_matrix(y2_test, predictions2_class)
print(con_mat)

[[26  3]
 [ 3  2]]


In [34]:
# Score model
print(classification_report(y2_test, predictions2_class))

              precision    recall  f1-score   support

         0.0       0.90      0.90      0.90        29
         1.0       0.40      0.40      0.40         5

    accuracy                           0.82        34
   macro avg       0.65      0.65      0.65        34
weighted avg       0.82      0.82      0.82        34



In [35]:
# Save model
# name2 = f"unshuffled-2q-out-{int(time.time())}"
# model.save(f"models/{name2}.h5")

### Predict on 2019

In [36]:
# Scale 2019 data 
scaled_X2_2019 = X2_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X2_2019 = reshape_data(scaled_X2_2019)

# Predict on 2019
pred_X2_2019 = model.predict_proba(reshaped_X2_2019)
print(f"2019Q1 No Recession Probability: {pred_X2_2019[0][0]}")
print(f"2019Q1 Recession Probability: {pred_X2_2019[0][1]}")
print(f"2019Q2 No Recession Probability: {pred_X2_2019[1][0]}")
print(f"2019Q2 Recession Probability: {pred_X2_2019[1][1]}")

2019Q1 No Recession Probability: 0.9999736547470093
2019Q1 Recession Probability: 2.6330559194320813e-05
2019Q2 No Recession Probability: 0.9999526739120483
2019Q2 Recession Probability: 4.734797767014243e-05


#### Predict on full X2

In [37]:
pred_X2_full = model.predict_classes(reshaped_X2_full)
X2_full_results = pd.DataFrame({"Predicted":pred_X2_full, "Actual":y2})
X2_full_results.loc[X2_full_results["Actual"]==1]
# X2_full_results.to_csv(f"resources/predictions/X2_NS_{int(time.time())}.csv")

,Predicted,Actual
quarter,,
1978Q4,1,1.0
1979Q1,1,1.0
1979Q2,1,1.0
1979Q3,1,1.0
1979Q4,0,1.0
1980Q4,1,1.0
1981Q1,1,1.0
1981Q2,1,1.0
1981Q3,1,1.0


### Train and predict on X3-Y3 data (recession 4 quarters out)

In [38]:
# Fit the model to the training data
model.fit(reshaped_X3_train_scaled, y3_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 106 samples, validate on 27 samples
Epoch 1/100
106/106 - 2s - loss: 0.3255 - acc: 0.8962 - val_loss: 0.7580 - val_acc: 0.8148
Epoch 2/100
106/106 - 2s - loss: 0.3756 - acc: 0.8962 - val_loss: 0.6379 - val_acc: 0.7407
Epoch 3/100
106/106 - 2s - loss: 0.3056 - acc: 0.9057 - val_loss: 0.6308 - val_acc: 0.7407
Epoch 4/100
106/106 - 2s - loss: 0.1118 - acc: 0.9245 - val_loss: 0.6836 - val_acc: 0.7778
Epoch 5/100
106/106 - 2s - loss: 0.2276 - acc: 0.8774 - val_loss: 0.5717 - val_acc: 0.7778
Epoch 6/100
106/106 - 2s - loss: 0.1770 - acc: 0.9245 - val_loss: 0.4998 - val_acc: 0.8148
Epoch 7/100
106/106 - 2s - loss: 0.1764 - acc: 0.9434 - val_loss: 0.4072 - val_acc: 0.7778
Epoch 8/100
106/106 - 3s - loss: 0.1026 - acc: 0.9528 - val_loss: 0.4192 - val_acc: 0.7778
Epoch 9/100
106/106 - 2s - loss: 0.0963 - acc: 0.9623 - val_loss: 0.4428 - val_acc: 0.8148
Epoch 10/100
106/106 - 2s - loss: 0.1491 - acc: 0.9340 - val_loss: 0.7020 - val_acc: 0.7778
Epoch 11/100
106/106 - 2s - loss: 0.1082 - a

Epoch 90/100
106/106 - 2s - loss: 0.0092 - acc: 1.0000 - val_loss: 1.6818 - val_acc: 0.7778
Epoch 91/100
106/106 - 2s - loss: 0.0092 - acc: 1.0000 - val_loss: 1.7202 - val_acc: 0.7778
Epoch 92/100
106/106 - 2s - loss: 0.0053 - acc: 1.0000 - val_loss: 1.7498 - val_acc: 0.7778
Epoch 93/100
106/106 - 1s - loss: 0.0311 - acc: 0.9906 - val_loss: 1.7398 - val_acc: 0.7778
Epoch 94/100
106/106 - 1s - loss: 0.0077 - acc: 1.0000 - val_loss: 1.7352 - val_acc: 0.7778
Epoch 95/100
106/106 - 2s - loss: 0.0121 - acc: 1.0000 - val_loss: 1.7087 - val_acc: 0.7778
Epoch 96/100
106/106 - 2s - loss: 0.0141 - acc: 1.0000 - val_loss: 1.6687 - val_acc: 0.7778
Epoch 97/100
106/106 - 1s - loss: 0.0082 - acc: 1.0000 - val_loss: 1.6722 - val_acc: 0.7778
Epoch 98/100
106/106 - 2s - loss: 0.0213 - acc: 0.9906 - val_loss: 1.7415 - val_acc: 0.7778
Epoch 99/100
106/106 - 1s - loss: 0.0051 - acc: 1.0000 - val_loss: 1.7864 - val_acc: 0.7778
Epoch 100/100
106/106 - 2s - loss: 0.0115 - acc: 1.0000 - val_loss: 1.8281 - val

In [39]:
# Validate model using test data
model_loss3, model_accuracy3 = model.evaluate(reshaped_X3_test_scaled, y3_test, verbose=2)

34/34 - 0s - loss: 1.1913 - acc: 0.7941


In [40]:
# Make predictions using test data
predictions3_class = model.predict_classes(reshaped_X3_test_scaled)

In [41]:
# Compare results
four_qtrs_out = pd.DataFrame({"Predicted":predictions3_class, "Actual":y3_test})
four_qtrs_out.loc[four_qtrs_out["Actual"]==1]

,Predicted,Actual
quarter,,
2006Q4,1,1.0
2007Q2,0,1.0
1989Q4,0,1.0
1979Q2,0,1.0
2007Q1,0,1.0


#### Confusion Matrix on X3-Y3 data (recession 4 quarters out)

In [42]:
# Create confusion matrix on X3 model
con_mat = confusion_matrix(y3_test, predictions3_class)
print(con_mat)

[[26  3]
 [ 4  1]]


In [43]:
# Score model
print(classification_report(y3_test, predictions3_class))

              precision    recall  f1-score   support

         0.0       0.87      0.90      0.88        29
         1.0       0.25      0.20      0.22         5

    accuracy                           0.79        34
   macro avg       0.56      0.55      0.55        34
weighted avg       0.78      0.79      0.78        34



In [44]:
# Save model
# name3 = f"unshuffled-4q-out-{int(time.time())}"
# model.save(f"models/{name3}.h5")

### Predict on 2019

In [45]:
# Scale 2019 data 
scaled_X3_2019 = X3_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X3_2019 = reshape_data(scaled_X3_2019)

# Predict on 2019
pred_X3_2019 = model.predict_proba(reshaped_X3_2019)
print(f"2019Q1 No Recession Probability: {pred_X3_2019[0][0]}")
print(f"2019Q1 Recession Probability: {pred_X3_2019[0][1]}")
print(f"2019Q2 No Recession Probability: {pred_X3_2019[1][0]}")
print(f"2019Q2 Recession Probability: {pred_X3_2019[1][1]}")

2019Q1 No Recession Probability: 0.9999973773956299
2019Q1 Recession Probability: 2.567917363194283e-06
2019Q2 No Recession Probability: 0.9999994039535522
2019Q2 Recession Probability: 5.892876515645185e-07


#### Predict on full X3

In [46]:
pred_X3_full = model.predict_classes(reshaped_X3_full)

# Preview results
X3_full_results = pd.DataFrame({"Predicted":pred_X3_full, "Actual":y3})
X3_full_results.loc[X3_full_results["Actual"]==1]

# Export results for graphing
# X3_full_results.to_csv(f"resources/predictions/X3_NS_{int(time.time())}.csv")

,Predicted,Actual
quarter,,
1978Q2,0,1.0
1978Q3,0,1.0
1978Q4,1,1.0
1979Q1,0,1.0
1979Q2,0,1.0
1980Q2,1,1.0
1980Q3,1,1.0
1980Q4,1,1.0
1981Q1,1,1.0
